In [1]:
import synapseclient
import pandas as pd
import json
import csv

In [2]:
def TapPerformance(TapPD1):
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonLeft', 'TappedButtonId'] = 0
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonRight', 'TappedButtonId'] = 1
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonNone', 'TappedButtonId'] = 2
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonLeft'] = 0
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonRight'] = 1
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonNone'] = 2
    TapPD1 = Artifactfixed(TapPD1)
    TapPerf= TapPD1.shape[0]
    return (TapPerf,TapPD1)

In [3]:
def Artifactfixed(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TapCoordinate[item]==TapCont.TapCoordinate[item+1]:
            if TapCont.TappedButtonId[item]==2:
                Ind[item]=False
            elif TapCont.TappedButtonId[item+1]==2:
                Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed

In [5]:

syn = synapseclient.Synapse()
syn.login('catarivera8', 'Crm1061709701')


## QUERY THE mPower PROJECT (syn4993293) FOR ALL OF THE TABLES
# See examples: https://python-docs.synapse.org/build/html/index.html 
# and https://www.synapse.org/#!Synapse:syn4993293/discussion/threadId=6785
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')

      
TapPD = list()
PathList = list()
LabelList = list()
PatientType = list()
PatientList = list()
Perform =list()
for offset in range(0,800,100):
    results = syn.tableQuery('SELECT * FROM syn5511439 LIMIT 100 OFFSET '+str(offset))
    file_map=syn.downloadTableColumns(results,['tapping_results.json.TappingSamples'])
    for file_handle_id, path in file_map.items():
        TapPD.append(pd.read_json(path))
        PathList.append(path)
        LabelList.append(file_handle_id)
        PatientType.append(TapTable['medTimepoint'][TapTable['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        Perform.append(TapPerformance(pd.read_json(path)))
        PatientList.append(TapTable['healthCode'][TapTable['tapping_results.json.TappingSamples'] == int(file_handle_id)])

Welcome, Catalina Rivera!



In [8]:
print(len(LabelList))
print(len(PatientList))

800
800


In [34]:
print(len(TapPD))
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')
    
TapTable.head()

7800


,ROW_ID,ROW_VERSION,recordId,healthCode,createdOn,appVersion,phoneInfo,accel_tapping.json.items,tapping_results.json.ButtonRectLeft,tapping_results.json.ButtonRectRight,tapping_results.json.endDate,tapping_results.json.startDate,tapping_results.json.TappingSamples,tapping_results.json.TappingViewSize,medTimepoint
0,82315,18,caf7a74b-0d2e-4c43-9233-ed5dc6e7b176,639e8a78-3631-4231-bda1-c911c1b169e5,1.425900e+12,"version 1.0, build 7",iPhone 6,5395782.0,"{{0, 191.5}, {104, 104}}","{{128, 191.5}, {104, 104}}",1.425900e+12,1.425900e+12,5395798.0,"{375, 603}",I don't take Parkinson medications
1,82316,18,94c95572-b405-49ca-9eb0-7426254990d5,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,1.425930e+12,"version 1.0, build 7",iPhone 5s (GSM),5403976.0,"{{0, 156.5}, {104, 104}}","{{128, 156.5}, {104, 104}}",1.425930e+12,1.425930e+12,5403991.0,"{320, 504}",I don't take Parkinson medications
2,82317,18,ebd99144-5b84-486e-818c-5f8daf8beebf,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5397281.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5397298.0,"{414, 672}",I don't take Parkinson medications
3,82318,18,8e32cff2-c8f9-4047-af57-411a98fc9116,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5395191.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5395211.0,"{414, 672}",I don't take Parkinson medications
4,82319,18,69851454-05b2-44f1-9248-299ca684b801,be43841c-d034-4705-88da-87e3536163b2,1.425930e+12,"version 1.0, build 7",iPhone 6,5390401.0,"{{0, 189}, {104, 104}}","{{128, 189}, {104, 104}}",1.425930e+12,1.425930e+12,5390413.0,"{375, 603}",I don't take Parkinson medications


In [20]:
df = pd.DataFrame({'A' : [5,6,3,4], 'B' : [1,2,3, 5]})
df

,A,B
0,5,1
1,6,2
2,3,3
3,4,5


In [22]:
df[df['A'] == 3]

,A,B
2,3,3
